# Basic Usage Examples

This notebook provides examples of the core functionality of `nbtest`. Notebooks that use `nbtest` must first load the extension as shown in the next cell.

In [1]:
%load_ext nbtest

Tags begin with the `@` symbol and should be legal Python identifiers (after the @). Tags found in the docstring (e.g. `@answer1`) of the cell are added to the cell cache. The cell can be re-run by test code so cells like `@answer1` should not assign variables if test code will modify them. 

In [2]:
# Create variables in the notebook
a = 100 
b = 200 

Here's an example of a solution cell.

In [3]:
"""@answer1"""

print("Hello World")
a < b

Hello World


True

You can introspect and run cells easily using the API for `nbtest`. 

In [4]:
import nbtest

answer1 = nbtest.get('@answer1')
print("Answer source:", repr(answer1.source))
print("Answer result:", type(answer1.result))
print("Answer result value:", answer1.result.result)


a = 200
b = 100 
result = answer1.run()
print("Re-run answer result:", result.result)

# run() doesn't affect the cache.
print("Original result value:", answer1.result.result)

Answer source: '"""@answer1"""\n\nprint("Hello World")\na < b\n'
Answer result: <class 'IPython.core.interactiveshell.ExecutionResult'>
Answer result value: True
Re-run answer result: False
Original result value: True


In [5]:
help(answer1)

Help on TagCacheEntry in module nbtest.tagcache object:

class TagCacheEntry(builtins.object)
 |  TagCacheEntry(result, shell)
 |
 |  Information about an executed cell.
 |
 |  Methods defined here:
 |
 |  __init__(self, result, shell)
 |      Create an entry.
 |
 |  run(self, push: Mapping = {}, capture: bool = True) -> Optional[nbtest.tagcache.CellRunResult]
 |      Run the contents of a cached cell.
 |
 |      push: Update variables in the notebook namespace with names and values in `push` before running the contents.
 |      capture: Set to `True` (the default) to capture stdout, stderr and output. If `False` run() returns `None`
 |
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |
 |  docstring
 |      The docstring of the cell or `None` if there is no docstring.
 |
 |  id
 |      The unique identifier of the notebook cell.
 |
 |  result
 |      The ExecutionResult from running the cell in IPython.
 |
 |  source
 | 

In [6]:
help(result)

Help on CellRunResult in module nbtest.tagcache object:

class CellRunResult(builtins.object)
 |  CellRunResult(stdout: str, stderr: str, outputs: list[typing.Any], result: Any) -> None
 |
 |  The result of calling run() on a TagCacheEntry
 |
 |  Methods defined here:
 |
 |  __eq__(self, other)
 |      Return self==value.
 |
 |  __init__(self, stdout: str, stderr: str, outputs: list[typing.Any], result: Any) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __annotations__ = {'outputs': list[typing.Any], 'result': typing.Any, ...
 |
 |  __dataclass_fi

## Running Notebook Cells 

The `run()` function returns a `CellRunResult` object that, by default, contains the outputs, stdout and stderr generated by running the cell. Re-running a cell with `run()` does not affect the cached copy that was run in the notebook.

In [7]:
result = answer1.run()
print("Cell stdout:", repr(result.stdout))
print("Cell stderr:", repr(result.stderr))
print("Cell result:", result.result)
print("Cell outputs:", result.outputs)

Cell stdout: 'Hello World\n'
Cell stderr: ''
Cell result: False
Cell outputs: [False]


The `run()` function can set variables in the notebook context to simplify testing.

In [8]:
print("1 < 2 ==", answer1.run({'a': 1, 'b': 2}).result)
print("2 < 2 ==", answer1.run({'a': 2, 'b': 2}).result)

1 < 2 == True
2 < 2 == False


By default the `run()` function captures stdout and stderr as well as any cell outputs. If you want output from running the cell to be processed the usual way in the notebook pass in `capture=False`.

In [9]:
answer1.run(capture=False)

Hello World


False

> When `capture` is `False` the `run()` function returns `None` instead of a `CellRunResult`. 


## The `%%testing` Cell Magic

The `%%testing` cell magic provides a wrapper for test code. `%%testing` cells render feedback for students. Tests run in a separate module namespace, just as they would for normal unit tests. Attributes listed after the testing cell magic are imported into the test namespace so they are available to test code. Imports in testing cells are not shared with the rest of the notebook.

Assertions are a convenient way to do simple checks.

In [10]:
%%testing @answer1

import ast 

# The `answer1` variable contains the cell cache entry 
assert ast.Lt in answer1.tokens, "I don't see less than!"
assert ast.Gt not in answer1.tokens, "I see greater than!"

Test cells always succeed, unless there's a problem with the test code. When tests fail students are shown your messages.

In [11]:
%%testing @answer1

# Failures don't stop the cell.
assert ast.Lt not in answer1.tokens, "The answer doesn't contain the a less than sign."

Since test code runs in a separate namespace it's required to use `shell.push()` to set variables in the notebook namespace. The `run()` function can also set variables in the notebook namespace.

In [12]:
%%testing @answer1

# The `shell` variable contains the interactive shell.
shell.push({'a': 1, 'b': 2}) 
assert answer1.run().result == True, "1 < 2 != True"

# `run()` optionally pushes variables too.
assert answer1.run({'a': 3, 'b': 2}).result == False, "3 < 2 != False"

If an attribute is missing the testing cell provides helpful feedback.

In [13]:
%%testing bogus 

# Bad params cause an error. 
pass

### Test Methods 

Functions that begin with `test` become test cases. Test cases are run and results are rendered below the cell. Test results use the docstring of test functions and assertion errors to inform the student how to fix their problems.

In [14]:
%%testing @answer1

# Functions that start with `test` become a `FunctionTestCase`
def test_lt(): 
    """Testing less than."""
    assert answer1.run({'a': 1, 'b': 2}).result == True, "1 < 2 != True"
    assert answer1.run({'a': 3, 'b': 2}).result == False, "3 < 2 != False"    

def test_broken_nodoc(): 
    assert answer1.run({'a': 1, 'b': 2}).result == False, "1 < 2 != True"

def test_broken_withdoc(): 
    """The test description is used to provide feedback to the learner. They will see this message when the test fails. It's useful to put a hint here."""
    assert answer1.run({'a': 1, 'b': 2}).result == False, "1 < 2 != True" 

Testing cell source is useful but solutions may contain variables, classes or functions. Those can be tested too.


In [15]:
"""@answer2"""

def add(a, b):
    """My solution function"""
    return a + b 

When solutions contain symbols they must be added to the testing magic to be present in the test namespace.

In [16]:
%%testing add

# Attributes must be listed to be present in the test namespace.
assert add.__doc__ is not None, "add() doesn't have a docstring."

def test_add():
    assert add(1,2) == 3, "1 + 2 != 3"
    assert add(-2,-2) == -4, "-2 + -2 != -4"
    assert add(1,-3) == -2, "1 + -3 != -2"

def test_err():
    """This is a description of the error that has happened."""
    assert add(0,"bogus"), "This is a bogus test."

### Unit Tests 

The test functions are based on Python's `unittest`. Any classes in a testing cell that derive form `unittest.TestCase` are automatically run.

In [17]:
%%testing add 

# Get the full power of TestCase (or write your own)

import unittest

class TestAdd(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(1,2), 3, "add(1,2)")
        self.assertEqual(add(-2,-2), -4, "add(-2,-2)")
        self.assertEqual(add(1,-3), -2, "add(-1,-3)")

    def test_badd_nodoc(self):
        self.assertEqual(add(1,2), 0, "add(1,2)")

    def test_badd_doc(self):
        """Bad test with documentation."""
        self.assertEqual(add(1,2), 0, "add(1,2)")

    def test_skip_me(self):
        self.skipTest("This test isn't relevant.")

The symbol `nbtest_attrs` is a dictionary that contains the attributes given to the `%%testing` cell magic.

In [18]:
%%testing @answer1 @answer2 add

from nbtest import nbtest_attrs

for attr in nbtest_attrs:
    print(f"Attribute: {attr} value: {nbtest_attrs[attr]}")

Attribute: answer1 value: <nbtest.tagcache.TagCacheEntry object at 0x768cf0fe7890>
Attribute: answer2 value: <nbtest.tagcache.TagCacheEntry object at 0x768cf0fe7530>
Attribute: add value: <function add at 0x768cf19ab380>


## Severity Signaling 

The `nbtest` module contains three decorators, `info`, `warning` and `error`. Decorating a test function with one of the decorators changes the style of the output feedback. Severity feedback can help students understand what to pay most attention to. 

In [19]:
%%testing @answer1

from nbtest import info, warning, error

# Functions that start with `test` become a `FunctionTestCase`
@info
def test_info(): 
    """There is something that requires a look but is not necessarily an error."""
    assert answer1.run({'a': 2, 'b': 2}).result == True, "1 < 2 != True"

@warning
def test_warning(): 
    """This demands attention and is probably an error but maybe not."""
    assert answer1.run({'a': 2, 'b': 2}).result == True, "1 < 2 != True"

@error
def test_error(): 
    """An error has happened."""
    assert answer1.run({'a': 2, 'b': 2}).result == True, "1 < 2 != True"

def test_also_error(): 
    """Unless otherwise specified, a failure is an error."""
    assert answer1.run({'a': 2, 'b': 2}).result == True, "1 < 2 != True"